In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
import numpy as np

In [ ]:
pip install timm

In [ ]:
pip install requests

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torchvision import transforms


In [ ]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm


In [ ]:
pip install flask

In [ ]:
import os
import json
from flask import Flask, request, jsonify, render_template

In [ ]:
model1 = tf.keras.models.load_model("D:\Mobilenet.h5")

In [ ]:
# Load the dataset
dataset_path = './Images'
dataset = datasets.ImageFolder(dataset_path, transform=transforms)

In [ ]:
def predict_dog(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(32, 32))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)

    prediction = model1.predict(img_array)
    print(prediction)
    return "Dog" if prediction[0][0] > 0.12 else "Not Dog"

image_path = "D:\img_4.jpg"
result = predict_dog(image_path)

In [ ]:
model_path = "D:/efficientnet_v01_trained.pth"

model = torch.load(model_path)
model.eval()  

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict(input_image: Image):
    input_tensor = transform(input_image)
    input_batch = input_tensor.unsqueeze(0)

    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
        

    _, predicted_idx = torch.max(output, 1)
    class_to_idx = dataset.class_to_idx
    idx_to_class = {idx: class_ for class_, idx in class_to_idx.items()}
    return idx_to_class[predicted_idx.item()] 

In [ ]:

if result != "Dog":
    raise ValueError("The prediction is not 'Dog'. Aborting further tests.")
else:
    app = Flask(__name__)

    model = torch.load(model_path, map_location='cpu')
    model.eval()  

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    dataset_path = os.path.join(".", "Images")
    dataset = datasets.ImageFolder(dataset_path, transform=transform)

    class_to_idx = dataset.class_to_idx
    idx_to_class = {idx: class_ for class_, idx in class_to_idx.items()}

    @app.route('/', methods=['GET', 'POST'])
    def home():
        if request.method == 'POST':
            file = request.files.get('image')
            if file is None:
                return jsonify({'error': 'no file'}), 400

            image = Image.open(file.stream)
            image = transform(image)
            image = image.unsqueeze(0)

            with torch.no_grad():
                output = model(image)

            _, predicted_idx = torch.max(output, 1)
            predicted_class = idx_to_class[predicted_idx.item()]
            class_idx, class_name = predicted_class.split("-")
            class_name = class_name.replace("_", " ")

            return jsonify({'predicted_class': class_name})
        else:
            return render_template('index.html')

    if __name__ == '__main__':
        app.run(port=8080)
